<a href="https://www.kaggle.com/code/daiqing2009/ocr-performance-analysis-on-license-plates?scriptVersionId=172543186" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# OCR Performance Analysis on LICENSE PLATES
This notebook replies on the magnificant job done of https://www.kaggle.com/code/aslanahmedov/automatic-number-plate-recognition 
, which trained a fine-tuned YOLO Model for license plate location in each picutre.
We utilize the that in the first part License plate images Preparation as the basis of our project to analyze the OCR performance on 
* different scales of data(i.e. license plate images)
* different (combination of) hardware


# License plate images Preparation



### Dataset Analysis

| Name | Number of picture(approximate) | Need Crop | Crop Method | Remark|
|--- | --- | --- | --- | --- |
|number-plate-detection| 250 | Y | coordinates in XML | high-resolution pic |
|car-plate - detection | 500 | Y | coordinates in XML | medium-resolution pic  |
|indian-vehicle-dataset|1,000 | Y | coordinates in XML | low-medium resolution pic, license plate no. in XML|
| stanford-cars-dataset|20,000 | Y | annotation in mat | details in mat file unknown |
| synthetic-turkish-license-plates | 100,000 | N | N/A | synthtic crated, license plate no. as file name |
| us-license-plates | 4,000 | N | N/A | high-resulution front-view license plates with states name/symbol |
| us-license-plates | 20,000 | N | N/A | low-medium resulution front-view license plates with states name/symbol |

Besides, there's a dataset called **license-plate-digits-classification-dataset** so enable us to train own number recognition NN if needed. 

## Generate licence plates pictures from datasets
Since threre're plenty of **front-view license plates pictures with no need to crop** , it is  reasonable to work on those before turn to those pictures needing crop. 

Here we will not only detect license plate but else extract test from it. We already done it before let’s repeat it again. We will write function. First from bounding box we need to take our x,y,w,h and extract ROI. I place it at top of our functions in drawing part.

# OPTICAL CHARACTER RECOGNITION(OCR) Performance Analysis

## Baseline TESSERACT OCR

Optical character recognition (OCR) software that is used to extract text from the image. Tesseract OCR have a python API and it is open source. Firstly, we will do installation of it. It pretty simple and depend on you OS. You can find manual and files to download for installation [here](https://guides.library.illinois.edu/c.php?g=347520&p=4121425).


### LIMITATIONS OF PYTESSERACT

Tesseract works best when there is a clean segmentation of the foreground text from the background. In practice, it can be extremely challenging to guarantee these types of setups. There are a variety of reasons you might not get good quality output from Tesseract like if the image has noise on the background. The better the image quality (size, contrast, lightning) the better the recognition result. It requires a bit of preprocessing to improve the OCR results, images need to be scaled appropriately, have as much image contrast as possible, and the text must be horizontally aligned. Tesseract OCR is quite powerful but does have the following limitations.

__Tesseract limitations summed in the list.__
<ul>
  <li>The OCR is not as accurate as some commercial solutions available to us.</li>
  <li>Doesn't do well with images affected by artifacts including partial occlusion, distorted perspective, and complex background.</li>
  <li>It is not capable of recognizing handwriting.</li>
  <li>It may find gibberish and report this as OCR output.</li>
  <li>If a document contains languages outside of those given in the -l LANG arguments, results may be poor.</li>  
  <li>It is not always good at analyzing the natural reading order of documents. For example, it may fail to recognize that a document contains two columns, and may try to join text across columns.</li>
  <li>Poor quality scans may produce poor quality OCR.</li>
  <li>It does not expose information about what font family text belongs to.</li>
</ul>


### EXTRACT NUMBER PLATE TEXT FROM IMAGE

Firstly, we will load our image and convert to array. Crop our bounding box with coordinates of it. We will identify region of interest (ROI) and have look at our cropped image *Figure 15*.

In [1]:
import cv2 
import pytesseract

# Path to the image file
path = "/kaggle/input/us-license-plates-image-classification/new plates/test/ALABAMA/3.jpg"
img = cv2.imread(path)

# extract the characters from the image
text = pytesseract.image_to_string(img)
print(text)

ModuleNotFoundError: No module named 'cv2'

With use of tesseract, we will extract the text from the mage.


## Compare the performance with Spark OCR
https://nlp.johnsnowlabs.com/docs/en/ocr

## Improve the performance by utilizing GPU

# Ultimate Challenge: Hand-made OCR optimized for license Plate Recognition (good to have)

# 